In [1]:
import numpy as np
import os
import matplotlib.pyplot as plt
from scipy.interpolate import UnivariateSpline
#from scipy.interpolate import InterpolatedUnivariateSpline
from scipy.interpolate import interp1d
from astropy.cosmology import Planck15
from datetime import datetime
from decimal import Decimal
from scipy import stats
from datetime import datetime
import subprocess
c = 299792458 #speed of light m s^-1
plt.rcParams['figure.figsize'] = [20, 10]

startTime = datetime.now()

In [2]:
from os.path import expanduser
home = expanduser("~")

In [3]:
home

'/home/cvargas'

In [4]:
libdir = home + '/sed_research'

def read_params(paramfile):
    params = {}
    params["Filters"] = {}
    with open(paramfile) as f:
        for line in f:
            #print line.split()
            if line[0] != "#":
                ls = line.split()
                params["Filters"][ls[0]] = {}
                params["Filters"][ls[0]]["lam_c"] = np.float(ls[1])
                params["Filters"][ls[0]]["delt_lam"] = np.float(ls[2])
                params["Filters"][ls[0]]["5sigma"] = np.float(ls[3])
                params["Filters"][ls[0]]["filename"] = libdir + "/Filters/" + ls[4]
                params["Filters"][ls[0]]["eazy_tag"] = np.float(ls[5])
    return params

In [5]:
class sed(object):
    "Spectral Energy Distribution Object"
    def __init__(self,paramfile,templatefile):
        params = read_params(paramfile)
        self.filters = params.get("Filters")
        template = templatefile
        self.loglam,self.lognuLnu = np.loadtxt(template,usecols=(0,1), delimiter=" ", unpack=True)
        self.nuLnu = np.power(10.,self.lognuLnu)
        self.lam = np.power(10.,self.loglam)
        #loading filters
        for Filter in self.filters:
            lam,trans = np.loadtxt(self.filters[Filter]["filename"],usecols =(0,1),delimiter=" ",unpack=True)
            #Pass wavelength from Angstrom to microns
            self.filters[Filter]["lam"] = lam*1e-4
            self.filters[Filter]["trans"] = trans
            
    def redshift(self,z):
        #Applying a redshift
        self.z = z
        self.nuFnu = (self.nuLnu/(4.*np.pi*Planck15.luminosity_distance(z).value**2))
        self.lam_new = self.lam*(1.+z)
        #to Jansky
        self.Fnu = self.nuFnu*(self.lam_new*1e-6/c)*1e-10*1e23
        #Interpolated nuFnu
        self.inuFnu = interp1d(self.lam_new,self.nuFnu)
        self.iFnu = interp1d(self.lam_new,self.Fnu)
        
    def photometry(self):
        self.SNR = []
        self.phot_err = []
        self.phot_c = []
        self.phot = []
        self.lam_eff = []
        self.lam_c = []
        self.lam_err = []
        self.tag = []
        self.eazytag = []
        self.det = [] #Detections of upper_limits
        #Gets the photometry of all the filters
        for Filter in self.filters:
            lam = self.filters[Filter]["lam"]
            trans = self.filters[Filter]["trans"]
            nuFnu = self.inuFnu(lam)
            Fnu = self.iFnu(lam)
            #Doing the integral for photometry - effective stimulus
            #This is the photometry
            phot_c = np.trapz(Fnu*trans*lam,lam)/np.trapz(trans*lam,lam)
            #phot_c = np.trapz(nuFnu/lam*trans,lam)/np.trapz(trans,lam)
            
            self.phot_c.append(phot_c)
            
            #Calculating sigma_flux in Jy
            
            fivesigma_mag = self.filters[Filter]["5sigma"]
            
            sigma_flux = (10**(-(fivesigma_mag + 48.6)/2.5)*1e23)/5.
            
            #Random realization
            phot = np.random.normal(loc=phot_c,scale=sigma_flux)
            
            if phot<0:
                phot=0.
            
            #self.phot.append(phot)
            
            #Calculating SNR
            SNR = phot/sigma_flux
            self.SNR.append(SNR)
            
            #print "{}    {}    {}    {}".format(Filter,sigma_flux,phot,SNR)
            
            if SNR>3.:
                self.det.append("det")
                self.phot_err.append(sigma_flux)
                self.phot.append(phot)
            #Treatment of non detections
            else:
                self.det.append("up_lim")
                #Adding the treatment of 3/2
                
                phot = (3./2.)*sigma_flux
                phot_err = (3./2.)*sigma_flux
                
                #Randomizing the value
                #phot = np.random.normal(loc=phot,scale=phot_err)
                self.phot.append(phot)
                self.phot_err.append(phot_err)
                
            #Doing the integral for effective wavelength
            lam_eff = np.trapz(Fnu*trans*lam*lam,lam)/np.trapz(Fnu*trans*lam,lam)
            self.lam_eff.append(lam_eff)
            self.lam_c.append(self.filters[Filter]["lam_c"])
            self.lam_err.append(self.filters[Filter]["delt_lam"]/2.)
            self.tag.append(Filter)
            self.eazytag.append(self.filters[Filter]["eazy_tag"])
            #print SNR
            
    def plot_sed(self,scale="normal",color="blue",linewidth=0.5):
        #Plotting the SED and the possible Photometry
        #plt.title("mysed; z={}".format(self.z))
        plt.ylabel(r"$F_{\nu}$[Jy]")
        plt.xlabel("$\lambda$ [$\mu$m]")
        if scale=="normal":
            plt.plot(self.lam_new,self.iFnu(self.lam_new),color=color,linewidth=linewidth,label="{}".format(self.z))
        if scale=="loglog":
            plt.loglog(self.lam_new,self.iFnu(self.lam_new),color=color,label="{}".format(self.z))
        
    def plot_filters(self,norm):
        for Filter in self.filters:
            lam = mysed.filters[Filter]["lam"]
            trans = mysed.filters[Filter]["trans"]*norm
            plt.plot(lam,trans,label = Filter)
            
    def export_phot(self,program="EAZY"):
        idx = np.argsort(np.asarray(self.lam_c))
        phot = np.asarray(self.phot)[idx]
        phot_err = np.asarray(self.phot_err)[idx]
        tag = np.asarray(self.tag)[idx]
        eazytag = np.asarray(self.eazytag)[idx]
        if program=="EAZY":
            return phot, phot_err, tag, eazytag 

In [6]:
parfile = libdir + '/params/nir.par'
tempfile = libdir + '/Templates/ARP220_loglam_lognuLnu.txt'

In [7]:
#Initializing here, so we don't have to call the class every time

z = 2.5
mysed = sed(parfile,tempfile)
mysed.redshift(z)
mysed.photometry()
phot, phot_err, tag, eazytag = mysed.export_phot(program="EAZY")


In [8]:
#Translation from name to filters on EAZY FILTER.RES
eazyfolder = home + "/eazy-photoz"
with open(eazyfolder + "/inputs/zphot.translate","w") as f:
    for i in range(len(eazytag)):
        f.write("f_{} F{}\n".format(str(tag[i]),str(int(eazytag[i]))))
        f.write("e_{} E{}\n".format(str(tag[i]),str(int(eazytag[i]))))

In [9]:
#Creating a Mock catalog
zspace = np.linspace(1.0,1.5,100) #Redshift grid

In [10]:
with open(home + "/eazy-photoz/inputs/eazy.cat","w") as f:
    L = "#id"
    for i in range(len(tag)):
        L+= " f_{} e_{}".format(tag[i],tag[i])
#    f.write(L + " z_spec" + "\n")
    f.write(L + "\n")
    L = "#id"
    for i in range(len(eazytag)):
        L+= " F{} E{}".format(np.int(eazytag[i]),np.int(eazytag[i]))
#    f.write(L + " z_spec" + "\n")
    f.write(L + "\n")
    for i in range(len(zspace)):
        mysed.redshift(zspace[i])
        mysed.photometry()
        phot, phot_err, tag, eazytag = mysed.export_phot(program="EAZY")
        L = "{}".format(i+1)
        for j in range(len(phot)):
            #EAZY format for Flux is microJansky
            L += "  " + "{:.5e}".format(Decimal(str(1e6*phot[j]))) + "  " + "{:.5e}".format(Decimal(str(1e6*phot_err[j])))
#        L+= " -1.0000"
        L+= "\n"
        f.write(L)
    print "done"

done


In [11]:
#Running EAZY from here
os.chdir(home + "/eazy-photoz/inputs/")
subprocess.call("../src/eazy -p zphot.param.default",shell=True)

0

In [12]:
zout = np.loadtxt(home + "/eazy-photoz/inputs/OUTPUT/photz.zout",comments="#",usecols=(14),unpack=True)

In [13]:
zout

array([0.7342, 0.7351, 0.7362, 0.7393, 0.7438, 0.7479, 0.7476, 0.7501,
       0.7545, 0.7587, 0.7597, 0.7613, 0.7624, 0.764 , 0.7666, 0.7743,
       0.8268, 0.8487, 0.862 , 0.8662, 0.8674, 0.8711, 0.8727, 0.8759,
       0.8794, 0.886 , 0.8954, 0.9018, 0.9039, 0.9073, 0.908 , 0.9093,
       0.9099, 0.914 , 0.9176, 0.9207, 0.9228, 0.9254, 0.9259, 0.9283,
       0.9303, 0.9302, 0.9349, 0.9374, 0.9461, 0.9442, 0.95  , 0.9525,
       0.966 , 0.9659, 0.9742, 0.9781, 0.9839, 0.9879, 0.991 , 0.9982,
       1.0008, 1.0035, 1.0045, 1.0079, 1.0115, 1.0163, 1.0198, 1.0246,
       1.0252, 1.0337, 1.0357, 1.036 , 1.0414, 1.0467, 1.0499, 1.0579,
       1.0582, 1.0627, 1.0665, 1.0667, 1.0723, 1.0745, 1.0776, 1.0803,
       1.0873, 1.0929, 1.0901, 1.0981, 1.103 , 1.1043, 1.1051, 1.1096,
       1.1095, 1.1138, 1.1165, 1.1213, 1.1245, 1.1291, 1.1355, 1.1337,
       1.142 , 1.14  , 1.14  , 1.1481])

In [14]:
plt.rcParams['figure.figsize'] = [10, 10]

0.8277806732673268 -0.07300984158415869


In [15]:
slope, intercept, r_value, p_value, std_err = stats.linregress(zspace,zout)
x = np.linspace(1.0,2.5,1000)
y = slope*x+intercept
plt.plot(x,y,color="red",label="linear_regress: slope = {}, intercept = {}".format(slope,intercept))
print slope,intercept

plt.plot(x,x,color="magenta",label="y=x")

plt.scatter(zspace,zout,c="blue",alpha=0.03,label="10000 sims")
plt.legend()
plt.show()

In [16]:
phot_test = np.array([1.70771e-01,1.91405e+00,5.69421e+00,3.25895e+01,8.45730e+01,1.19146e+02,1.57714e+02])*1e-6
phot_err_test = np.array([7.44628e-02,3.79339e-02,2.65289e-02,4.55096e-02,5.64738e-01,9.86226e-01,9.14601e-01])*1e-6
lam_c = np.array([2.9928e+03,4.5573e+03,6.0013e+03,7.9960e+03,1.2289e+04,1.6444e+04,2.2124e+04])*1e-4

In [25]:
%matplotlib auto

Using matplotlib backend: Qt5Agg


In [29]:
lam2,Flam2 = np.loadtxt(home + "/eazy-photoz/templates/vb/Arp220_template_norm.sed",unpack=True)
nuFnu2 = Flam2*lam2
lam2um = lam2*1e-4

nu2 = c/(lam2*1e-10)

Fnu2 = nuFnu2/nu2*1e23*160./6.1e15

plt.loglog(lam2um,Fnu2,"r-",markersize=0.5)
plt.xlabel(r"$\lambda [\mu m]$")
plt.ylabel(r"$F\nu$ $[Jy]$")

z = 0.0185
mysed = sed(parfile,tempfile)
mysed.redshift(z)
mysed.photometry()

ax = plt.subplot(111)
ax.set_xscale("log", nonposx='clip')
ax.set_yscale("log", nonposy='clip')

plt.errorbar(mysed.lam_c, mysed.phot, xerr=mysed.lam_err, yerr=mysed.phot_err,fmt="bo",ecolor="blue")
plt.plot(mysed.lam_eff, mysed.phot_c, "ro")
mysed.plot_sed(scale="normal",color="red")

plt.show()

/home/cvargas/miniconda2/lib/python2.7/site-packages/matplotlib/cbook/deprecation.py:107: MatplotlibDeprecationWarning: Adding an axes using the same arguments as a previous axes currently reuses the earlier instance.  In a future version, a new instance will always be created and returned.  Meanwhile, this warning can be suppressed, and the future behavior ensured, by passing a unique label to each axes instance.
  warnings.warn(message, mplDeprecation, stacklevel=1)


In [ ]:
print datetime.now() - startTime